Load in data tables and format them into F Data Grids

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
root = os.getcwd()
dataroot = r'C:\Users\George\Documents\Python Scripts\FDA\Letters\Data'

In [4]:
#locate and read data files
os.chdir(dataroot)
xdata,ydata = pd.read_csv('xdata_abc.csv'),pd.read_csv('ydata_abc.csv')
xdata.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,0.806452,0.765161,0.734194,0.865806,0.708387,0.749677,0.744516,0.716129,0.705806,0.760000,...,0.685161,0.721290,0.569032,0.790968,0.651613,0.775484,0.762581,0.584516,0.736774,0.579355
1,0.806452,0.680000,0.685161,0.809032,0.674839,0.726452,0.710968,0.698065,0.625806,0.718710,...,0.587097,0.695484,0.507097,0.762581,0.633548,0.741935,0.734194,0.522581,0.705806,0.543226
2,0.765161,0.594839,0.602581,0.723871,0.630968,0.667097,0.659355,0.669677,0.561290,0.638710,...,0.509677,0.664516,0.421935,0.734194,0.584516,0.708387,0.705806,0.458065,0.643871,0.486452
3,0.680000,0.486452,0.538065,0.630968,0.576774,0.594839,0.600000,0.623226,0.499355,0.550968,...,0.440000,0.625806,0.378065,0.700645,0.548387,0.669677,0.672258,0.393548,0.576774,0.440000
4,0.615484,0.421935,0.486452,0.532903,0.543226,0.522581,0.532903,0.587097,0.432258,0.489032,...,0.378065,0.587097,0.310968,0.646452,0.509677,0.625806,0.628387,0.334194,0.527742,0.401290


To use FDataGrids, each column should be of equal length.
To acheive this the readdata class will be used to get the smoothed data each of equal vector legnth.

In [6]:
from Letters.readdata import letterData

In [7]:
letterdata = letterData('xdata_abc.csv','ydata_abc.csv')

In [9]:
#return to original directory
os.chdir(root)